In [1]:
#importing required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
s = Service("C:\msedgedriver.exe")
driver = webdriver.Edge(service= s)
driver.get("https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=2022%2F23;trophy=89;type=season")
#page url we need to scrape
driver.maximize_window()

In [3]:
soup = BeautifulSoup(driver.page_source, 'lxml') #parsing the webpage

In [4]:
all_data = soup.find_all("tr",{"class":"data1"}) #class continer which contains all the required data

In [5]:
Match_id= []
Team1= []
Team2= []
for data in all_data:
    n= data.find_all("td",{"nowrap":"nowrap"})
    x = [i.text for i in n]
    Team1.append(x[0])
    Team2.append(x[1])
    Match_id.append(x[5])
#scraping match_id, Team1 and Team2    

In [6]:
teams=[]
for i in range(0,45):
    k= Team1[i]+" vs "+Team2[i]
    teams.append(k)   
#concatenating teams    

In [ ]:
bowling_df= pd.DataFrame()
vs_teams = []
total_match_id = []
for i in range(0,45): #iterating till the total number of matches played
    driver.find_element(By.XPATH, f'//*[@id="ciHomeContentlhs"]/div[3]/div/table[1]/tbody/tr[{i+1}]/td[7]/a').click()
    time.sleep(1)
    data= pd.read_html(f"{driver.current_url}")
    
    try:
        first_team_bowler= data[1]
        first_team_bowler= first_team_bowler[first_team_bowler[['BOWLING','O','M','R','W','ECON','0s','4s','6s','WD','NB']].nunique(axis=1) != 1]
        second_team_bowler= data[3]
        second_team_bowler= second_team_bowler[second_team_bowler[['BOWLING','O','M','R','W','ECON','0s','4s','6s','WD','NB']].nunique(axis=1) != 1]
        bowling_df= pd.concat([bowling_df,first_team_bowler])
        bowling_df= pd.concat([bowling_df,second_team_bowler])
        
        for value in range(len(first_team_bowler)+len(second_team_bowler)):
            vs_teams.append(teams[i])
            total_match_id.append(Match_id[i])
            
    except:
        pass
    driver.back()
    time.sleep(1)
    
  

In [ ]:
#len(vs_teams)
#len(bowling_df)
len(total_match_id)

In [ ]:
bowling_df

In [ ]:
bowling_df.insert(loc=0, column="Match",value= vs_teams, allow_duplicates= False)
bowling_df.insert(loc=1, column="Match_id",value= total_match_id, allow_duplicates= False)
#added new columns to bowling dataframe

In [ ]:
bowling_df.head(5)

In [ ]:
bowling_df.reset_index(drop=True, inplace=True)
#resetting index

In [ ]:
bowling_df.head(5)

In [ ]:
#bowling_df.to_csv("C:\\Users\\Gurjeet\\Desktop\\End to End Cricket Analytics Project\\Webscraped Files\\fact_bowling.csv",encoding="utf-8-sig")

In [ ]:
#driver.close()